# 🚀 Scaling the Patch Camelyon

Welcome to the second part of the Patch Camelyon project! In this section, we’ll take your deep learning pipeline to the next level by focusing on scalability and maintainability. We’ll leverage PyTorch Lightning and Hydra to build a robust pipeline, integrate logging for comprehensive monitoring, and set up job submission for efficient training on a cluster.


### Notebook Structure 📚

1. Drawbacks of the Current Approach 📉
2. Configuring with Hydra ⚙️
3. Creating a Proper ML Project with Version Control 📦
4. Integrating Logging with MLFlow 📊
5. Job Submission for Scalable Training 🖥️

**Authors:**<br>
&copy; 2024; Matěj Pekár, Vít Musil

Before we start, let's install the required libraries:

In [ ]:
pip install hydra-core

## 1. Drawbacks of the Current Approach 📉

While developing in a single notebook is convenient for quick experimentation, it has its limitations, especially as your project grows. Let’s dive into some of the key challenges we face with this approach:

### Scalability Challenges

- **Resource Management**: Our current setup doesn’t automatically free up resources after job completes, leading to potential wastage.

- **Long-Running Jobs**: There’s no **checkpointing**, meaning if a long training job gets interrupted, we lose all progress. This makes it difficult to handle more complex models or larger datasets efficiently.

- **Distributed Training**: Our current setup doesn’t support distributed training, which is essential for leveraging multiple GPUs or nodes to speed up training on large datasets.


### Traceability Issues

- **Parameter Tracking**: It’s challenging to track which parameters we’ve tried, making it difficult to understand what worked and what didn’t. We lack a clear history of our experiments, leading to potential redundancy or missed insights.

- **Results Comparison**: Comparing results from different runs is cumbersome without a systematic way to log and retrieve this information.


### Reproducibility Challenges

- **Version Control**: Without proper version control, it’s hard to ensure that the code we’re running today is the same as what we’ll run tomorrow. This makes it difficult to share our work or reproduce results later.

- **Randomness Control**: We haven’t set a seed for random operations, which means our results might vary from run to run, making it hard to trust the outcomes.

    > **Note**: Not all [algorithms](https://pytorch.org/docs/stable/notes/randomness.html#avoiding-nondeterministic-algorithms) in PyTorch are deterministic, but setting a seed can help ensure reproducibility to some extent.



By highlighting these drawbacks, we can see why it’s important to transition to a more scalable, traceable, and reproducible approach. This sets the stage for the next steps in our project, where we’ll address some of these issues and build a more robust machine-learning pipeline.

## 2. Configuring with [Hydra](https://hydra.cc) ⚙️

Hydra is a powerful configuration management tool that makes it easy to compose and override configurations, offering a clean and maintainable approach to setting up your deep learning pipeline. By separating configuration from code, Hydra allows you to store settings in YAML files, making it simple to experiment with different hyperparameters and configurations. Instead of hard-coding parameters, you can adjust them in a YAML file, making your pipeline more flexible and easier to manage.

Hydra is built on top of OmegaConf so if you are looking for a more detailed explanation of the configuration system, you can check out the [OmegaConf documentation](https://omegaconf.readthedocs.io/en/2.3_branch/).

### Getting Started with Hydra

Before we dive into using Hydra, let's get the necessary configuration files in place. Run the following cell to download and store them in the `hydra_configs` directory:

In [2]:
from io import BytesIO
from zipfile import ZipFile

import requests


r = requests.get(
    "https://youtrack.rationai.cloud.e-infra.cz/api/files/225-202?sign=MTcyNTU4MDgwMDAwMHwxLTh8MjI1LTIwMnx2UnZDMksyM1dQNExDTHlTdDhxbV9nOGZVY2tBS0lHMEtzTkFBRjlBUk1VDQo&updated=1725401287969&forceDownload=true"
)
with ZipFile(BytesIO(r.content)) as zip_ref:
    zip_ref.extractall(".")

In the following examples, we’ll show you the basics of using Hydra. For the exercises ahead, you’ll work with YAML files stored in the `hydra_configs` folder. If you get stuck use the documentation or ask ChatGPT for help.

But before we dive in, we’ll define a simple helper method. Since we’re working in a Jupyter notebook environment, this method will simulate running the code from the command line with Hydra, allowing you to see how it all fits together.

In [2]:
import hydra
from omegaconf import DictConfig


def parse_hydra_conf(
    config_name: str,
    config_path: str = "hydra_configs",
    cli_args: list[str] | None = None,
) -> DictConfig:
    with hydra.initialize(version_base=None, config_path=config_path):
        return hydra.compose(config_name=config_name, overrides=cli_args)

<style>
code {
    color: purple;
    background-color: #f0f0f0;
}
</style>

<div class="alert alert-success code">
    <h3>TASK 2.1: Defining A List and Dictionary</h3>
    <p>
        In this task, you’ll be working with the <code>hydra_configs/task1.yaml</code> file. Your goal is to define: <ul> <li>A list</li> <li>A dictionary</li> <li>A list of dictionaries</li> </ul> Ensure that everything is set up correctly so that the assertions run without any errors.
    </p>
</div>

In [ ]:
from omegaconf import OmegaConf


config = parse_hydra_conf("task1.yaml")
print(OmegaConf.to_yaml(config))

assert config.learning_rates == [1e-3, 1e-4, 1e-5]
assert config.loss_weights == {"dice": 1.0, "ce": 0.9}
assert config.optimizers == [
    {"name": "sgd", "learning_rate": 1e-4},
    {"name": "adam", "learning_rate": 1e-3, "decay": 1e-4},
]

<style>
code {
    color: purple;
    background-color: #f0f0f0;
}
</style>

<div class="alert alert-success code">
    <h3>TASK 2.2: Changing, Overriding and Deleting Configuration</h3>
    <p>
        In this task, you’ll be working with the <code>hydra_configs/task2.yaml</code> file. Your objective is to override specific configuration settings directly from the command line (by defining the <code>cli_args</code> variable in this notebook). <br><br> Specifically, you’ll need to: <ul> <li>Set the <code>checkpoint</code> parameter to <code>None</code> (you can do this one directly inside the YAML file) .</li> <li>Set <code>trainer.max_epochs</code> to <strong>10</strong>.</li> <li>Add <code>data.batch_size</code> with a value of <strong>32</strong>.</li>
        <li>Delete the <code>trainer.logger</code>.</li> </ul> This task will help you understand how to dynamically adjust configurations using Hydra.
    </p>
</div>

In [ ]:
cli_args: list[str] = [
    "checkpoint=null",
    "trainer.max_epochs=10",
    "+data.batch_size=32",
    "~trainer.logger",
]

config = parse_hydra_conf("task2.yaml", cli_args=cli_args)
print(OmegaConf.to_yaml(config))

assert config.checkpoint is None
assert config.trainer.max_epochs == 10
assert config.data.batch_size == 32
assert not hasattr(config.trainer, "logger")

<style>
code {
    color: purple;
    background-color: #f0f0f0;
}
</style>

<div class="alert alert-success code">
    <h3>TASK 2.3: Interpolation</h3>
    <p>
        In this task, you’ll be working with the <code>hydra_configs/task3.yaml</code> file, where you'll leverage interpolation to avoid redundancy and keep your configuration clean and maintainable. Complete the TODOs in the YAML file to interpolate the values correctly.
    </p>
</div>

In [ ]:
config = parse_hydra_conf("task3.yaml")
config.batch_size = 64
config.data.val.transforms = config.data.val.transforms[:-1]
print(OmegaConf.to_yaml(config))


assert config.data.test.transforms == ["rotate", "flip"]
assert config.data.batches == 64

<style>
code {
    color: purple;
    background-color: #f0f0f0;
}
</style>

<div class="alert alert-success code">
    <h3>TASK 2.4: Instantiation</h3>
    <p>
        In this task, you’ll be modifying the <code>hydra_configs/task4/default.yaml</code> file and use the <code>hydra_configs/task4/example.py</code>. Your objective is to define define model named "unet" with backbone which has 3 classes. We encourage you to also use the interpolation from the previos task. This task will help you understand how you can instantiate classes and functions directly from config files. This is useful when you want to define your model, dataset, or any other object in a configuration file.
    </p>
</div>

In [ ]:
from hydra_configs.task4.example import Backbone, Model


config = parse_hydra_conf("default.yaml", config_path="hydra_configs/task4")
print(OmegaConf.to_yaml(config))

model = hydra.utils.instantiate(config.class_example)
function = hydra.utils.instantiate(config.function_example)

assert isinstance(model, Model)
assert model.name == "unet"
assert isinstance(model.backbone, Backbone)
assert model.backbone.num_classes == 3


assert isinstance(function, Model)
assert function.name == "unet"
assert isinstance(function.backbone, Backbone)
assert function.backbone.num_classes == 3

<style>
code {
    color: purple;
    background-color: #f0f0f0;
}
</style>

<div class="alert alert-success code">
    <h3>TASK 2.5: Config Groups</h3>
    <p>
        In this task, you’ll be editing only the <code>hydra_configs/task5/default.yaml</code> file. The goal is to streamline your configurations by organizing them into groups. Here’s what you need to do: <ul> <li>Start with the <code>base.yaml</code> as your foundation.</li> <li>Link the <code>/data/datasets/cityscapes.yaml</code> configuration to <code>data.train</code>.</li> <li>Set the <code>data.train.transforms</code> to use the transformations defined in <code>/data/transforms/train.yaml</code>.</li> <li>Override the <code>data.train.path</code> to point to <code>"/some/data"</code>.</li> <li>Finally, make the <code>model.backbone</code> required.</li> </ul> This task will help you understand how to group configurations effectively and set up defaults that make your pipeline more flexible and easier to manage. By organizing your configs in this way, you’re setting yourself up for smoother and more controlled experimentation. 
    </p>
</div>

In [ ]:
cli_args: list[str] = ["model/backbone=vgg16"]

config = parse_hydra_conf(
    "default.yaml", config_path="hydra_configs/task5", cli_args=cli_args
)
print(OmegaConf.to_yaml(config))

assert config.data.batch_size == 64
assert config.model.backbone.name == "vgg16"

assert config.data.train.type == "cityscapes"
assert config.data.train.path == "/some/data"
assert config.data.train.transforms == ["rotate", "flip", "normalize"]


cli_args: list[str] = ["model/backbone=resnet18"]

config = parse_hydra_conf(
    "default.yaml", config_path="hydra_configs/task5", cli_args=cli_args
)
print(OmegaConf.to_yaml(config))

assert config.model.backbone.name == "resnet18"

## 3. Creating a Proper ML Project with Version Control 📦

Let's dive into the next step of professionalizing your machine learning workflow! In this section, you'll set up a robust version control system to keep your project organized, reproducible, and collaborative.


### Step-by-Step Guide
1. **Get Started with Version Control**:

    First, create an account on a version control platform like GitHub, GitLab, or Bitbucket. This is where you’ll manage your project and track its history.

2. **Set Up Your Project**:

    Create a new **PUBLIC** repository by either:
    - Directly importing our [template repository](https://gitlab.ics.muni.cz/rationai/digital-pathology/templates/machine-learning.git)
    - Or by cloning it to your local machine and pushing it to your repository.

3. **Customize the Template**:

    Now, it’s time to fill in the template with what you’ve learned so far. Aim to extract all configurable elements into separate configuration files for better flexibility. Keep your configurations simple and intuitive—avoid overcomplicating things!
    
    > ⚠️ Tip: The image you pulled includes a sample dataset to test your pipeline. You’ll find it at `/mnt/data/PatchCamelyon`. This path is consistent with the cluster environment, so no need to worry about data paths.


### Dependency Management Made Easy with [PDM](https://pdm-project.org/en/latest/)

We’re using PDM for dependency management. Unlike pip, PDM offers a lock file, ensuring stable and reproducible development. Plus, it’s more flexible than Poetry, allowing you to run custom scripts easily.

Here are some handy commands to get you started:
```bash
pdm install # install dependencies
pdm add <package> # add a new package
pdm remove <package> # remove a package
pdm run <file> # run a file
pdm <script> # run a script from pyproject.toml
```

### Running Your Pipeline

To kick off your training, validation, testing, or prediction processes, simply use the following predifined scripts (defined in the `pyproject.toml`):
```bash
pdm train
pdm val
pdm test
pdm predict
```
>**_NOTE:_** Before running the pipeline you have to export your `MLFLOW_TRACKING_USERNAME` and `MLFLOW_TRACKING_PASSWORD` as environment variables because logging to MLFlow is enabled by default.


### What Your Project Should Look Like

By the end of this task, your project structure should resemble this:
```bash
.
├── LICENSE
├── README.md
├── configs
│   ├── data
│   │   ├── datasets
│   │   │   └── patch_camelyon
│   │   │       ├── test.yaml
│   │   │       ├── train.yaml
│   │   │       └── val.yaml
│   │   └── transforms
│   │       ├── test.yaml
│   │       └── train.yaml
│   ├── default.yaml
│   ├── hydra
│   │   └── default.yaml
│   ├── logger
│   │   └── mlflow.yaml
│   └── model
│       └── backbone
│           └── ...
├── patch_camelyon
│   ├── __init__.py
│   ├── data
│   │   ├── __init__.py
│   │   ├── data_module.py
│   │   └── datasets
│   │       ├── __init__.py
│   │       └── patch_camelyon.py
│   ├── main.py
│   ├── modeling
│   │   ├── __init__.py
│   │   ├── backbone
│   │   │   ├── __init__.py
│   │   │   └── ...
│   │   └── binary_classifier.py
│   ├── patch_camelyon_model.py
│   └── typing.py
├── pdm.lock
└── pyproject.toml
```

By the time you complete this task, you’ll have a fully configured ML project that’s easy to manage, share, and extend.

## 4. Integrating Logging with [MLFlow](https://mlflow.org) 📊

Now that we’ve set up our project, it’s time to supercharge it with experiment tracking. We’ll be using **MLFlow**, one of the most powerful open-source platforms for managing the end-to-end machine learning lifecycle.

You can access our self hosted MLFlow instance at [https://biomedai-summer-mlflow.dyn.cloud.e-infra.cz](https://biomedai-summer-mlflow.dyn.cloud.e-infra.cz).

### Step 1: Running Your First Logged Training Run

Let’s kick things off by running a training script to see MLFlow in action. By default, your run should automatically log key elements like:

- **Loss** and **metrics** during training.

- **Git information** to track the code version.

- **Console logs** for debugging and insights.


> Tip: Run your training script a few times with different settings. This will allow you to explore the MLFlow UI, compare different runs, and visualize the results. It’s a great way to see how your model evolves with each experiment!


### Step 2: Adding Checkpointing for Robust Training

Imagine your training process gets interrupted—no one likes to start over from scratch, right? Let’s safeguard your progress by enabling **checkpointing**. This allows you to save your model’s state after each epoch so you can resume training if something goes wrong.

Here’s how:

1. **Add a Checkpoint Callback**:

    Use the [ModelCheckpoint callback](https://lightning.ai/docs/pytorch/stable/api/lightning.pytorch.callbacks.ModelCheckpoint.html) in your PyTorch Lightning `Trainer`. Configure it to track the **validation loss** and save checkpoints.
    > **NOTE**: You should configure this entirely through Hydra.

2. **Verify Checkpoint Saving**:

    Run your training again and check if checkpoints are being saved as artifacts in MLFlow after each epoch. These will be super handy for resuming training or for later evaluation!


### Step 3: Resuming Training from a Checkpoint

To test out your checkpointing setup, try resuming your training from the last saved checkpoint:

- Set the `checkpoint` parameter in your configuration to the MLFlow URI of the checkpoint you want to resume from.

- Run your training script again and watch as it picks up right where it left off!


### Step 4: Evaluating the Model on the Test Set

Once you’ve logged your checkpoints and explored how to resume from them, it’s time to evaluate your model:

- Use the logged checkpoint to perform evaluation on your test set.

- Check MLFlow to ensure that the results are logged properly.

By integrating MLFlow into your pipeline, you’re not just training a model—you’re creating a rich, trackable experiment history. This will help you and your team to easily reproduce results, compare different approaches, and ensure that you’re always moving forward.

## 5. Job Submission for Scalable Training 🖥️

To harness the full power of cluster computing, you'll need to submit your job to the cluster environment. But first, let's ensure your code is ready for prime time!




### Step 1: Push Your Code to a Remote Repository


Before you can submit your job, make sure your code is pushed to a **PUBLIC** remote repository (like GitHub, GitLab, or Bitbucket). This step is crucial for the accessibility of your code in a cluster environment, as well as the reproducibility of your experiments. By logging the commit hash, you can always trace back and reproduce the exact version of the code used in your experiment. This is a best practice in machine learning, helping ensure that your results are reliable and can be replicated.

### Step 2: Download the Helm Chart

Next, you'll need a [Helm chart](https://helm.sh/docs/topics/charts/) to submit your job to the cluster. Helm charts are like blueprints for deploying applications on Kubernetes, and they’ll help you manage your training jobs efficiently.

Run the following cell to download and store the Helm chart in the `job` directory:

In [4]:
r = requests.get(
    "https://youtrack.rationai.cloud.e-infra.cz/api/files/225-203?sign=MTcyNTU4MDgwMDAwMHwxLTh8MjI1LTIwM3wzNjJIZFpUY3JzLXU0R19fQ3pDOXBENlFJZ3NPb0h4NmlyMHQ1MDR3emU4DQo&updated=1725401404972&forceDownload=true"
)
with ZipFile(BytesIO(r.content)) as zip_ref:
    zip_ref.extractall(".")

### Step 3: Configure Your Job

Next, let’s set up your job configuration. This is where you'll specify your repository, the exact code version you want to run, and the commands for your training job.

1. Open the `job/values.yaml` File. This is your job’s configuration file, where you’ll set up the details needed to run your model on the cluster.

2. Replace the value of `git.repository` field with the URL of your own repository.

3. In the `git.commit` field, enter the specific commit hash or branch name you want the job to use.

4. In the `script` field, specify the commands for your training job, like `pdm install` for dependencies and `pdm train` to start training.



### Step 4: Submit Your Job

Once your configuration is ready, it’s time to launch your training job on the cluster.

Run the following command to submit your job:

In [ ]:
!helm install -f job/values.yaml job-<your_name> job

If you need to cancel or delete your job, use the following command:

In [ ]:
!helm uninstall job-<your_name>

Once your job is submitted and resources become available, it will start running on the cluster. You can track its progress through the [MLFlow UI](https://biomedai-summer-mlflow.dyn.cloud.e-infra.cz).